In [11]:
import pandas as pd
import requests
import os

# =============================================================================
# CONFIGURATION - Modifiez ces constantes selon vos besoins
# =============================================================================

# Identifiants OpenFoodFacts (obligatoires)
USERNAME = ""
PASSWORD = ""

# Données mode d'élevage à uploader
# 2 colonnes utilisées dans chaque csv : 'barcode' et 'tag'
# colonne 'tag' de new_breedings ---> ajoute un tag de catégorie au produit OFF ('en:cage-chicken-eggs')
# colonne 'tag' de new_quantities ---> remplace la quantité du produit OFF ('12 pcs')
DATA_PATH = "../data/"
FILE_NAME_BREEDINGS = "new_breedings_mai_2025.csv"
FILE_NAME_QUANTITIES = "new_quantities_mai_2025.csv"

# API OpenFoodFacts
OFF_API_URL = "https://world.openfoodfacts.org/api/v0/product/"
OFF_WRITE_URL = "https://world.openfoodfacts.org/cgi/product_jqm2.pl"

# =============================================================================


# Upload des modes d'élevage

In [12]:
new_breedings = pd.read_csv(os.path.join(DATA_PATH, FILE_NAME_BREEDINGS), sep=";")
new_breedings

,barcode,tag
0,5400208005190,en:organic-eggs
1,5701607583013,en:organic-eggs
2,8033576311098,en:organic-eggs
3,8033622750932,en:organic-eggs
4,8420415012689,en:organic-eggs
...,...,...
392,3578402323258,en:chicken-eggs
393,3250393378515,en:chicken-eggs
394,3448601000000,en:chicken-eggs
395,3700468700791,en:chicken-eggs


In [ ]:



def test_off_product(barcode, new_tag, user_id, password):
    """
    Teste la lecture d'un produit et ajoute un tag de catégorie via l'API OpenFoodFacts sans écraser les autres tags.
    """
    # Lecture produit
    read_url = f"{OFF_API_URL}{barcode}.json"
    response = requests.get(read_url)
    if response.status_code != 200:
        raise Exception(f"Erreur lors de la lecture du produit {barcode} : HTTP {response.status_code}")

    product_data = response.json()
    print(f"Produit: {product_data.get('product', {}).get('product_name', 'Inconnu')}")
    print(f"Catégories avant ajout: {product_data.get('product', {}).get('categories_tags', [])}")

    # Ajout du tag sans écraser
    payload = {
        "code": barcode,
        "add_categories": new_tag,
        "user_id": user_id,
        "password": password,
        "comment": "Ajout automatique via script",
    }
    write_response = requests.post(OFF_WRITE_URL, data=payload)
    if write_response.status_code != 200:
        raise Exception(f"Erreur lors de l'écriture du produit {barcode} : HTTP {write_response.status_code}")

    print(f"Tag '{new_tag}' ajouté pour le produit {barcode}.")


def add_tags_from_dataframe(df, user_id, password):
    """
    Ajoute les tags de catégorie pour tous les produits dans un DataFrame sans écraser les catégories existantes.
    Le DataFrame doit contenir les colonnes 'barcode' et 'tag'.
    """
    for _, row in df.iterrows():
        barcode = row['barcode']
        tag = row['tag']
        payload = {
            "code": barcode,
            "add_categories": tag,
            "user_id": user_id,
            "password": password,
            "comment": "Ajout automatique via script",
        }
        response = requests.post(OFF_WRITE_URL, data=payload)
        if response.status_code != 200:
            print(f"Erreur pour le produit {barcode} : HTTP {response.status_code}")
        else:
            print(f"Tag '{tag}' ajouté pour le produit {barcode}.")


# Exemple d'utilisation

# Exemple : test d'un seul produit
if input('chargement produit test (o/n)') == 'o':
    test_off_product("3276570018128", "en:cage-chicken-eggs", USERNAME, PASSWORD)

# Ajout des produits de la df
if input(f'chargement les données de la dataframe new_breedings sur OFF  ({new_breedings.shape[0]} produits) : saisir mass_upload') == 'mass_upload':
    add_tags_from_dataframe(new_breedings, USERNAME, PASSWORD)


# Upload des quantités

In [17]:
new_quantities = pd.read_csv(os.path.join(DATA_PATH, FILE_NAME_QUANTITIES), sep=";")
new_quantities

,barcode,quantité,tag
0,3025950000389,4,4 pcs
1,8002790086044,4,4 pcs
2,8008110007867,4,4 pcs
3,8003636004536,4,4 pcs
4,8006071950437,4,4 pcs
...,...,...,...
588,3298624300010,30,30 pcs
589,3300755362307,30,30 pcs
590,6111262585557,30,30 pcs
591,13251212512000082794,90,90 pcs


In [ ]:
def test_off_product(barcode, new_tag, user_id, password):
    """
    Teste la lecture d'un produit et ajoute un tag de catégorie via l'API OpenFoodFacts sans écraser les autres tags.
    """
    # Lecture produit
    read_url = f"{OFF_API_URL}{barcode}.json"
    response = requests.get(read_url)
    if response.status_code != 200:
        raise Exception(f"Erreur lors de la lecture du produit {barcode} : HTTP {response.status_code}")

    product_data = response.json()
    print(f"Produit: {product_data.get('product', {}).get('product_name', 'Inconnu')}")
    print(f"Quantité avant ajout: {product_data.get('product', {}).get('quantity', [])}")

    # Ajout du tag sans écraser
    payload = {
        "code": barcode,
        "quantity": new_tag,
        "user_id": user_id,
        "password": password,
        "comment": "Ajout automatique via script",
    }
    write_response = requests.post(OFF_WRITE_URL, data=payload)
    if write_response.status_code != 200:
        raise Exception(f"Erreur lors de l'écriture du produit {barcode} : HTTP {write_response.status_code}")

    print(f"Tag '{new_tag}' ajouté pour le produit {barcode}.")


def add_tags_from_dataframe(df, user_id, password):
    """
    Ajoute les tags de catégorie pour tous les produits dans un DataFrame sans écraser les catégories existantes.
    Le DataFrame doit contenir les colonnes 'barcode' et 'tag'.
    """
    for _, row in df.iterrows():
        barcode = row['barcode']
        tag = row['tag']
        payload = {
            "code": barcode,
            "quantity": tag,
            "user_id": user_id,
            "password": password,
            "comment": "Ajout automatique via script",
        }
        response = requests.post(OFF_WRITE_URL, data=payload)
        if response.status_code != 200:
            print(f"Erreur pour le produit {barcode} : HTTP {response.status_code}")
        else:
            print(f"Tag '{tag}' ajouté pour le produit {barcode}.")


# Exemple d'utilisation

# Ajout des produits de la df
if input(f'chargement les données de la dataframe new_quantities sur OFF  ({new_quantities.shape[0]} produits) : saisir mass_upload') == 'mass_upload':
    add_tags_from_dataframe(new_quantities, USERNAME, PASSWORD)